In [ ]:
import ee
import geemap
ee.Initialize()

In [ ]:
def distanceFilter(pts,distance):
    withinDistance = distance; ##go with pretty far apart at first

    ## From the User Guide: https:#developers.google.com/earth-engine/joins_spatial
    ## add extra filter to eliminate self-matches
    distFilter = ee.Filter.And(ee.Filter.withinDistance(**{
      'distance': withinDistance,
      'leftField': '.geo',
      'rightField': '.geo', 
      'maxError': 1
    }), ee.Filter.notEquals(**{
      'leftField': 'system:index',
      'rightField': 'system:index',

    }));
    
    distSaveAll = ee.Join.saveAll(**{
                  'matchesKey': 'points',
                  'measureKey': 'distance'
    });
    # Apply the join.
    spatialJoined = distSaveAll.apply(pts, pts, distFilter);

    # Check the number of matches.
    # We're only interested if nmatches > 0.
    spatialJoined = spatialJoined.map(lambda f: f.set('nmatches', ee.List(f.get('points')).size()) );
    spatialJoined = spatialJoined.filterMetadata('nmatches', 'greater_than', 0);

    # The real matches are only half the total, because if p1.withinDistance(p2) then p2.withinDistance(p1)
    # Use some iterative logic to clean up
    def unpack(l): 
        return ee.List(l).map(lambda f: ee.Feature(f).id())

    def iterator_f(f,list):
        key = ee.Feature(f).id()
        list = ee.Algorithms.If(ee.List(list).contains(key), list, ee.List(list).cat(unpack(ee.List(f.get('points')))))
        return list
    
    ids = spatialJoined.iterate(iterator_f,ee.List([]))
    ##print("Removal candidates' IDs", ids);

    # Clean up 
    cleaned_pts = pts.filter(ee.Filter.inList('system:index', ids).Not());
    return cleaned_pts



In [ ]:
import math
import numpy as np

# function that will do all the math thats needed for a given scenario
def do_the_math(total_area, scenario, size_classes, trt_unit_ratios):
    """Returns the amount of treatment units needed per size class and the radii of the kernel needed for each size class to make the shapes tiered treatment landscape in EE
    # args:
    # total_area (int): total area of aoi to consider treatments in acres
    # scenario (float): percent of total area to be treated in the given scenario, expressed as float [0,1]
    # size_classes (list): list of unique treatment sizes in acres in descending order of size (e.g. [100,50,10] ) 
    # trt_unit_ratios (list): ratio of size classes to make up the given pct treated in the scenario, expressed as a list of floats equaling scenario (i.e. if scenario = 0.6, trt_unit_ratios could be [0.2,0.3,0.1] )
                                must match order of size_classes
    """
    if round(sum(trt_unit_ratios),1) != scenario:
        raise RuntimeError(f"sum of trt_unit_ratios does not equal scenario: ({scenario})")
    if len(size_classes) != len(trt_unit_ratios):
        raise RuntimeError(f"size_classes and trt_unit_ratios must be of same length")
    
    # solve for the radii of the treatment units for each treatment size in meters, radius is passed to an EE function to create the shapes
    acres_to_sqm = [int(round(size_i*4027)) for size_i in size_classes] # convert to sq meters for each size class in acreage
    radii_circles = [int(round(math.sqrt(acreage/math.pi))) for acreage in acres_to_sqm] # solve for r: A = pi(r^2)
    
    # next solve for how many of each treatment units you'd need per size class for each scenario (pct total to treat and the proportions among the size classes)
    area_of_scenario = total_area*scenario # compute total area to be treated per treatment scenario
    area_per_size_class = [int(round(area_of_scenario*ratio)) for ratio in trt_unit_ratios]
    units_per_size_class = [int(round(i)) for i in list(np.divide(area_per_size_class,size_classes))]
    return units_per_size_class, radii_circles

In [ ]:
total_area=245278 #acres
scenario = 0.6
size_classes = [100,50,10] # acres
trt_unit_ratios = [0.2,0.2,0.2]

units,radii = do_the_math(total_area=total_area,scenario=scenario,size_classes=size_classes,trt_unit_ratios=trt_unit_ratios)
print('target units per size class:',units,'\n','radii per treatment unit size:',radii)

In [ ]:
def treatments(units,radii):
    seed = 5155115
    ptsBig = ee.Image.constant(1).clip(aoi).sample(aoi,30,'EPSG:3857',None,units[0]+(units[0]*2),seed,True,4,True) 
    ptsBig = distanceFilter(ptsBig,radii[0]*2.1).limit(units[0])
    #print('Big trt points',ptsBig.size().getInfo())
    areasBig = ptsBig.reduceToImage(['constant'],ee.Reducer.first()).unmask(0).clip(aoi).reduceNeighborhood(ee.Reducer.max(),ee.Kernel.octagon(radii[0],'meters')).eq(1)

    areasBigmask = areasBig.distance(ee.Kernel.euclidean(radii[0]*2,'meters')).gte(0).Not().unmask(1)

#     Map.addLayer(ptsBig,{},'big pts')
#     Map.addLayer(areasBig, pal, 'big treatments')
#     Map.addLayer(areasBigmask,pal,'areasBigmask')
    
    ptsMedium = areasBigmask.selfMask().sample(aoi,30,'EPSG:3857',None,units[1]+(units[1]*2),seed,True,4,True).map(lambda f: f.set('nd',1))
    ptsMedium = distanceFilter(ptsMedium,radii[1]*2.1).limit(units[1])
    #print('Medium trt points',ptsMedium.size().getInfo())
    areasMedium = ptsMedium.reduceToImage(['nd'],ee.Reducer.first()).unmask(0).clip(aoi).reduceNeighborhood(ee.Reducer.max(),ee.Kernel.octagon(radii[1],'meters')).eq(1)

    areasMediummask = areasMedium.distance(ee.Kernel.euclidean(radii[1]*2,'meters')).gte(0).Not().unmask(1)
    blendmask = areasBigmask.add(areasMediummask).lte(1).Not().unmask(1)

#     Map.addLayer(ptsMedium,{},'medium pts')
#     Map.addLayer(areasMedium, pal, 'medium treatments')
#     Map.addLayer(areasMediummask,pal,'areasMedium mask')
#     Map.addLayer(blendmask,pal,'blendmask')
    
    ptsSmall = blendmask.selfMask().sample(aoi,30,'EPSG:3857',None,units[2]+(units[2]*2),seed,True,4,True).map(lambda f: f.set('nd',1))
    ptsSmall = distanceFilter(ptsSmall,radii[2]*2.1).limit(units[2])
    #print('Small trt points',ptsSmall.size().getInfo())
    areasSmall = ptsSmall.reduceToImage(['nd'],ee.Reducer.first()).unmask(0).clip(aoi).reduceNeighborhood(ee.Reducer.max(),ee.Kernel.octagon(radii[2],'meters')).eq(1)
    
#     Map.addLayer(ptsSmall, {}, 'small points')
#     Map.addLayer(areasSmall, pal, 'small treatments')
    
    blendTreatments = areasBig.add(areasMedium).add(areasSmall).gte(1)
    
#     Map.addLayer(blendTreatments, pal,'final treatment landscape')

    return ee.Image(blendTreatments)

In [ ]:
Map = geemap.Map()

parcels = ee.FeatureCollection("projects/aff-treatments/assets/AllParcels_forSIG")
NIparcels = ee.FeatureCollection("projects/aff-treatments/assets/AllNIPParcels_forSIG")
fireshed = ee.FeatureCollection("projects/aff-treatments/assets/SonoraFireshed")
aoi = fireshed
Map.addLayer(parcels,{},'parcels')
Map.addLayer(NIparcels,{},'NIP parcels')
Map.addLayer(aoi,{},'fireshed')
Map.centerObject(aoi,12)

pal = {'min':0,'max':1,'palette':['black','white']}

trt_img = treatments(units,radii)

Map.addLayer(trt_img, pal, 'final trt landscape')

Map

In [ ]:
## OLD ##
# # Acreage area calcs for kernel radius
# # Total acreage of fireshed = 245278 acres
# # 60% treated = 147,166.8
# # 50
# #10 acre, 50 acre 100 acre, calculations for 30m/px scale composing a square area:
# # 10 acres = 40468 m^2 = 6.7 30m pixels each side, 45 pixels total
# # 50 acres = 202343 m^2 = 15 30m pixels each side, 225 pixels total
# # 100 acres = 404686 m^2 = 21.2 30m pixels each side, 450 pixels total

# # AOI is 432 km^2 = 106750 acres
# # how many of each treatment unit
# # 106750*0.1= 10675 acres / 100 = 106 100ac treatments
# # 106750*0.2 = 21350 acres / 50 = 427 50ac treatments
# # 106750*0.3 = 21350 acres / 10 = 2135 10ac treatments
# pal = {'min':0,'max':1,'palette':['black','white']}
# pts100 = ee.Image.constant(1).clip(aoi).sample(aoi,30,'EPSG:3857',None,106+(106*2),90210,True,4,True) 
# pts100 = distanceFilter(pts100,11*30*2).limit(106)
# #print('100ac points',pts100)

# areas100 = pts100.reduceToImage(['nd'],ee.Reducer.first()).unmask(0).clip(aoi).reduceNeighborhood(ee.Reducer.max(),ee.Kernel.octagon(11)).eq(1)

# areas100mask = areas100.distance(ee.Kernel.euclidean(8,'pixels')).gte(0).Not().unmask(1)

# pts50 = areas100mask.selfMask().sample(aoi,30,'EPSG:3857',None,427+(427*2),90210,True,4,True).map(lambda f: f.set('nd',1))
# pts50 = distanceFilter(pts50,7*30*2).limit(427)
# #print('50ac points',pts50)
# areas50 = pts50.reduceToImage(['nd'],ee.Reducer.first()).unmask(0).clip(aoi).reduceNeighborhood(ee.Reducer.max(),ee.Kernel.octagon(7.5)).eq(1)

# areas50mask = areas50.distance(ee.Kernel.euclidean(5,'pixels')).gte(0).Not().unmask(1)
# blendmask = areas100mask.add(areas50mask).lte(1).Not().unmask(1)

# pts10 = blendmask.selfMask().sample(aoi,30,'EPSG:3857',None,2135+(2135*2),8558,True,4,True).map(lambda f: f.set('nd',1))
# pts10 = distanceFilter(pts10,3.5*30*2).limit(2135)
# #print('10ac points',pts10.size().getInfo())
# areas10 = pts10.reduceToImage(['nd'],ee.Reducer.first()).unmask(0).clip(aoi).reduceNeighborhood(ee.Reducer.max(),ee.Kernel.octagon(3.5)).eq(1)

# blendTreatments = areas100.add(areas50).add(areas10).gte(1)

# Map.addLayer(pts100,{},'pts100ac')
# Map.addLayer(areas100, pal, '100ac treatments')
# #Map.addLayer(areas100mask,pal,'areas100mask')
# Map.addLayer(pts50,{},'pts50ac')
# Map.addLayer(areas50, pal, '50ac treatments')
# #Map.addLayer(areas50mask,pal,'areas50 mask')
# #Map.addLayer(blendmask,pal,'blendmask')
# Map.addLayer(pts10)
# Map.addLayer(areas10, pal, '10ac treatments')
# Map.addLayer(blendTreatments, pal,'blended treatments')

# # issues - points are still overlapping previously treated areas
# Map